In [1]:
import json
import requests

In [2]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display

In [3]:
def submit_to_ars(m):
    submit_url = 'https://ars.transltr.io/ars/api/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'https://arax.ncats.io/?source=ARS&id={message_id}')
    return message_id

In [4]:
def retrieve_ars_results(mid):
    message_url = f'https://ars.transltr.io/ars/api/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    for child in j['children']:
        if child['status'] == 'Done':
            childmessage_id = child['message']
            child_url = f'https://ars.transltr.io/ars/api/messages/{childmessage_id}'
            child_response = requests.get(child_url).json()
            try:
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except:
                nresults=0
        else:
            nresults = 0
        print( child['status'], child['actor']['agent'],nresults )
    return results

In [5]:
def printjson(j):
    print(json.dumps(j,indent=4))

def print_json(j):
    printjson(j)

In [6]:
def name_lookup(text):
    url= f'https://name-resolution-sri.renci.org/lookup?string={text}&offset=0&limit=10'
    response = requests.post(url)
    printjson(response.json())

In [7]:
def one_hop_message(subject_category,object_category,predicate=None,subject_id = None, object_id = None):
    query_graph = {
    "nodes": {
        'a':{
            "category": subject_category
        },
        'b':{
            "category": object_category
        }
    },
    "edges": {
        'ab':{
            "subject": "a",
            "object": "b",
            #"predicate": predicate
        }
    }
    }
    if predicate is not None:
        query_graph['edges']['ab']['predicate'] = predicate
    if subject_id is not None:
        query_graph['nodes']['a']['id'] = subject_id
    if object_id is not None:
        query_graph['nodes']['b']['id'] = object_id
    message = {"message": {"query_graph": query_graph},'knowledge_graph':{"nodes": [], "edges": [],}, 'results':[]}
    return message

In [12]:
x = one_hop_message("biolink:ChemicalSubstance","biolink:PhenotypicFeature",subject_id="CHEBI:17754")
printjson(x)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:ChemicalSubstance",
                    "id": "CHEBI:17754"
                },
                "b": {
                    "category": "biolink:PhenotypicFeature"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}


# Broaden to other diseases

**What are the inborn errors of metabolism? (MONDO:0019052)**

In [8]:
subclass_message = one_hop_message('biolink:Disease','biolink:Disease',predicate='biolink:subclass_of',object_id = "MONDO:0019052")
printjson(subclass_message)
subclass_id = submit_to_ars(subclass_message)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:Disease"
                },
                "b": {
                    "category": "biolink:Disease",
                    "id": "MONDO:0019052"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b",
                    "predicate": "biolink:subclass_of"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}
https://arax.ncats.io/?source=ARS&id=9381dd4b-4f26-42fb-b3e0-9bd4997acd51


# Look at processes?

In [9]:
glycerol_process_message = one_hop_message('biolink:ChemicalSubstance','biolink:BiologicalProcessOrActivity',predicate='biolink:related_to',subject_id = "CHEBI:17754")
printjson(glycerol_process_message)
glycerol_process_id = submit_to_ars(glycerol_process_message)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:ChemicalSubstance",
                    "id": "CHEBI:17754"
                },
                "b": {
                    "category": "biolink:BiologicalProcessOrActivity"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b",
                    "predicate": "biolink:related_to"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}
https://arax.ncats.io/?source=ARS&id=c2d60d0a-8c1e-468c-a8a1-a8f4105ad6fa


In [10]:
gk_process_message = one_hop_message('biolink:Gene','biolink:BiologicalProcessOrActivity',predicate='biolink:related_to',subject_id = "UniProtKB:P32189")
printjson(gk_process_message)
glycerol_process_id = submit_to_ars(gk_process_message)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:Gene",
                    "id": "UniProtKB:P32189"
                },
                "b": {
                    "category": "biolink:BiologicalProcessOrActivity"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b",
                    "predicate": "biolink:related_to"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}
https://arax.ncats.io/?source=ARS&id=eb6a3613-6ca8-4485-a97b-995ad7532b60
